In [2]:
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np

In [3]:
outfiles = glob.glob("/home/murez/out/*.out")

outfiles

[]

In [10]:
titlename_dict = {'/home/murez/out/rmat1_4.out':"rmat1_top5",
 '/home/murez/out/rmat3_4.out':"rmat3_top5",
 '/home/murez/out/web-Stanford.out':"web-Stanford",
 '/home/murez/out/rmat2_4.out':"rmat2_top5",
 '/home/murez/out/roadNet-CA.out':"roadNet-CA",
 '/home/murez/out/com-orkut.out':"com-orkut",
 '/home/murez/out/soc-LiveJournal1.out':"soc-LiveJournal1",
 '/home/murez/out/rmat1.out':"rmat1",
 '/home/murez/out/rmat3.out':"rmat3",
 '/home/murez/out/rmat2.out':"rmat2"}

for i in range(len(outfiles)):
    try:
        testfile = outfiles[i]
        threads = []
        m = {}
        timer = {}
        with open(testfile, 'r') as f:
            # thread = 0
            for line in f.readlines():
                if "omp_thread" in line:
                    thread = int(line.split(':')[-1])
                    threads.append(thread)
                if "MTEPS" in line and "Avg" not in line:
                    mteps = float(line.split(':')[-1])
                    if thread not in m.keys():
                        m[thread] = [mteps]
                    else:
                        m[thread].append(mteps)
                if "Trial Time" in line:
                    t = float(line.split(':')[-1])
                    if thread not in timer.keys():
                        timer[thread] = [t]
                    else:
                        timer[thread].append(t)

        d = {}

        for thread in m.keys():
            d[thread] = []
            for i in range(len(m[thread])):
                if timer[thread][i]>=0.001 and m[thread][i] >=5:
                    d[thread].append((timer[thread][i],m[thread][i]))

        x = np.random.randn(100, 8)

        all_timelist = []
        threadlist = np.array(list(d.keys()))
        print(threadlist)
        for thread in d.keys():
            all_timelist.append([x[1] for x in d[thread]])

        all_timelist = np.array(all_timelist)

        timelist_min = all_timelist.min(1)

        timelist_max = all_timelist.max(1)

        timelist_avg = all_timelist.mean(1)

        timelist_std = all_timelist.std(1)


            # mtepslist = [x[1] for x in d[thread]]
        plt.errorbar(threadlist, timelist_avg, timelist_std, fmt='ok', lw=3)
        plt.errorbar(threadlist, timelist_avg, [timelist_avg - timelist_min, timelist_max -     timelist_avg],
                     fmt='.k', ecolor='gray', lw=1)
        plt.plot(threadlist, timelist_avg)
        plt.xlabel("cores")
        plt.ylabel("MTEPS")
        plt.title(titlename_dict[testfile])
        file_name = testfile.split('.')[0]+'.MTEPS.png'

        plt.savefig(file_name)
        plt.cla()
        plt.clf()
    except:
        pass